<a href="https://colab.research.google.com/github/endophenotype/Spark/blob/main/Spark_%D0%A3%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81%D0%B5%D0%BA%D1%86%D0%B8%D0%BE%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Следующий код использует reduceByKey операцию над парами
ключ-значение, чтобы подсчитать, сколько раз каждая строка текста
встречается в файле

In [ ]:
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[1]'
conf = SparkConf().setAppName(appName).setMaster(master)
sc = spark.sparkContext
lines = sc.textFile("/content/P.txt")
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
for element in counts.collect():
 print(element)

('Python', 4)
('python', 2)


Широковещательные переменные создаются из переменной v путем вызова
SparkContext.broadcast(v). Широковещательная переменная представляет
собой оболочку вокруг v, и ее значение можно получить, вызвав value метод.
Код ниже показывает это:

In [ ]:
from pyspark import SparkContext, SparkConf
master = 'local[*]'
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
broadcastVar = sc.broadcast([1, 2, 3])
print(broadcastVar)
broadcastVar.value

[1, 2, 3]

В приведенном ниже коде показан аккумулятор, используемый для сложения
элементов массива:

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
accum = sc.accumulator(0)
sc.parallelize([1, 2, 3, 4]).foreach(lambda x: accum.add(x))
accum.value


10

Интерфейс AccumulatorParam имеет два метода: zero для предоставления
«нулевого значения» для вашего типа данных и addInPlace для сложения двух
значений вместе. Например, предположим, что у нас есть Vector класс,
представляющий математические векторы, мы могли бы написать:

In [ ]:
from pyspark.accumulators import AccumulatorParam
class VectorAccumulatorParam(AccumulatorParam):
  def zero(self, initialValue):
    return [0.0] * len(initialValue)
  def addInPlace(self, v1, v2):
    v1 += v2
    return v1
vecAccum = sc.accumulator([1, 2, 3], VectorAccumulatorParam())
vecAccum.value

[1, 2, 3]

In [ ]:
accum = sc.accumulator(0)
def g(x):
  global vecAccum
  accum.add(x)
  return f(x)
rdd = sc.parallelize(vecAccum.value)
rdd.map(g)


PythonRDD[9] at RDD at PythonRDD.scala:53

In [ ]:
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[*]'
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
nums = sc.parallelize([1, 2, 3, 4, 5])
print(nums.collect())
bytwo = nums.map(lambda x: x + 2)
print(bytwo.collect())
squared = nums.map(lambda x: x * x)
print(squared.collect())

[1, 2, 3, 4, 5]
[3, 4, 5, 6, 7]
[1, 4, 9, 16, 25]


Spark имеет как стандартный, так и пользовательский разделитель. Это означает, что когда вы создаете RDD, вы можете позволить Spark установить количество разделов или указать его явно. Количество разделов по умолчанию зависит от источника данных, размера кластера и доступных ресурсов. В большинстве случаев разделение по умолчанию будет работать нормально, но если вы опытный программист Spark, вы можете предпочесть явно указать количество разделов с помощью RDD.repartition, RDD.coalesce() или DataFrame.coalesce(). функция.
Spark предлагает несколько функций для управления секционированием. Вы можете использовать RDD.repartition(numPartitions), чтобы вернуть новый RDD, который имеет ровно numPartitions разделов. Эта функция может увеличивать или уменьшать уровень параллелизма в RDD, как показано в следующем примере:


In [ ]:
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[*]'
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 5)
print('rdd.getNumPartitions() ',rdd.getNumPartitions())
print('sorted(rdd.glom().collect()) ', sorted(rdd.glom().collect()))
print('len(rdd.repartition(4).glom().collect()) ', len(rdd.repartition(4).glom().collect()))
rdd = rdd.repartition(4).glom()
print('rdd.getNumPartitions() ',rdd.getNumPartitions())
print('sorted(rdd.glom().collect()) ', sorted(rdd.glom().collect()))
print('len(rdd.repartition(2).glom().collect()) ', len(rdd.repartition(2).glom().collect()))
rdd = rdd.repartition(2).glom()
print('rdd.getNumPartitions() ',rdd.getNumPartitions())
print('sorted(rdd.glom().collect()) ', sorted(rdd.glom().collect()))

rdd.getNumPartitions()  5
sorted(rdd.glom().collect())  [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]
len(rdd.repartition(4).glom().collect())  4
rdd.getNumPartitions()  4
sorted(rdd.glom().collect())  [[[1, 2]], [[3, 4]], [[5, 6, 7, 8]], [[9, 10]]]
len(rdd.repartition(2).glom().collect())  2
rdd.getNumPartitions()  2
sorted(rdd.glom().collect())  [[[[3, 4]]], [[[5, 6, 7, 8], [9, 10], [1, 2]]]]
